# Ejercicios Vivo Clase 2


## Resumen estadístico de columnas numéricas seleccionadas
Obtenga un resumen estadístico de las columnas 'AMT_INCOME_TOTAL', 'CNT_CHILDREN' y 'CNT_FAM_MEMBERS' usando el DataFrame 'application_record'.

In [ ]:
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Ejercicio Vivo Clase 2")\
        .getOrCreate()

In [ ]:
from zipfile import ZipFile
import requests

with open("./credit_cards_data.zip", "wb") as f:
    f.write(requests.get("https://github.com/engcarlosperezmolero/resources_and_tools/blob/main/data/csv/credit_cards_data.zip?raw=true").content)

zip_ref = ZipFile(f"/content/credit_cards_data.zip")
zip_ref.extractall()
zip_ref.close()

application_record = spark.read.csv('/content/credit_cards_data/application_record.csv', header=True, inferSchema=True)
credit_record = spark.read.csv('/content/credit_cards_data/credit_records_toy.csv', header=True, inferSchema=True)

In [ ]:
from pyspark.sql.functions import col, mean, stddev, count, format_number

# Función para calcular el resumen estadístico y formatear
def summary_stats(column):
    return (application_record.agg(mean(column).alias('mean'), stddev(column).alias('stddev'), count(column).alias('count')).select(format_number(col('mean'), 2).alias('mean'), format_number(col('stddev'), 2).alias('stddev'), col('count')))

# Aplicando la función a las columnas seleccionadas y mostrando el resultado
summary_stats('AMT_INCOME_TOTAL').show()
summary_stats('CNT_CHILDREN').show()
summary_stats('CNT_FAM_MEMBERS').show()

+----------+----------+------+
|      mean|    stddev| count|
+----------+----------+------+
|187,524.29|110,086.85|438557|
+----------+----------+------+

+----+------+------+
|mean|stddev| count|
+----+------+------+
|0.43|  0.72|438557|
+----+------+------+

+----+------+------+
|mean|stddev| count|
+----+------+------+
|2.19|  0.90|438557|
+----+------+------+



## Media de ingresos agrupados por género
Calcula la media de ingresos ('AMT_INCOME_TOTAL') agrupados por género ('CODE_GENDER') en el DataFrame 'application_record'.

In [ ]:
from pyspark.sql.functions import avg

# Calcular la media de ingresos agrupados por género y mostrar el resultado
application_record.groupBy('CODE_GENDER').agg(avg('AMT_INCOME_TOTAL').alias('mean_income')).show()

+-----------+------------------+
|CODE_GENDER|       mean_income|
+-----------+------------------+
|          F|174523.04091044018|
|          M| 214086.6388684194|
+-----------+------------------+



## Conteo de créditos por estado y género
Realice un join de los DataFrames 'application_record' y 'credit_record' por la columna 'ID' y encuentre el conteo de cada estado de crédito ('STATUS') agrupado por género ('CODE_GENDER').

In [ ]:
from pyspark.sql.functions import count

# Realizar el join de los DataFrames por ID
joined_df = application_record.join(credit_record, on='ID')

# Calcular conteo de créditos por estado y género
conteo_creditos_estado_genero = joined_df.groupBy('CODE_GENDER', 'STATUS').agg(count('STATUS').alias('count_status'))

# Mostrar resultado
conteo_creditos_estado_genero.show()

+-----------+------+------------+
|CODE_GENDER|STATUS|count_status|
+-----------+------+------------+
|          M|     C|      108080|
|          M|     1|        3217|
|          F|     C|      221456|
|          M|     4|          86|
|          M|     3|         108|
|          F|     4|         128|
|          M|     X|       50672|
|          M|     5|         718|
|          F|     2|         510|
|          M|     2|         291|
|          F|     1|        5530|
|          F|     X|       95278|
|          F|     5|         809|
|          F|     3|         178|
|          F|     0|      194962|
|          M|     0|       95692|
+-----------+------+------------+

